# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
pwd

'/Users/tupran/Documents/Data/GitHub/GuidedCapstone/DataWrangling'

**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [3]:
df = pd.read_csv('/Users/tupran/Documents/Data/GitHub/GuidedCapstone/data/step3_output.csv', index_col=0)
df.head()

,Name,state,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,cluster
0,Eaglecrest Ski Area,Alaska,2600,1540,1200,0,0.0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.00000,90.0,0.0,0
1,Hilltop Ski Area,Alaska,2090,294,1796,0,0.0,0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,0
2,Sunrise Park Resort,Arizona,11100,1800,9200,0,0.0,0,1,2,...,800.0,80.0,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,1
3,Yosemite Ski & Snowboard Area,California,7800,600,7200,0,0.0,0,0,0,...,88.0,0.0,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,1
4,Boreal Mountain Resort,California,7700,500,7200,0,0.0,0,1,1,...,380.0,200.0,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [4]:
dfs = df.drop('state', axis=1)
dfo = pd.get_dummies(df['state'], prefix='state')
df = pd.concat([dfs,dfo], axis=1)

In [5]:
df

,Name,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,Eaglecrest Ski Area,2600,1540,1200,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hilltop Ski Area,2090,294,1796,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Sunrise Park Resort,11100,1800,9200,0,0.0,0,1,2,3,...,0,0,0,0,0,0,0,0,0,0
3,Yosemite Ski & Snowboard Area,7800,600,7200,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Boreal Mountain Resort,7700,500,7200,0,0.0,0,1,1,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Tyrol Basin,1160,300,860,0,0.0,0,0,0,3,...,0,0,0,0,0,0,0,0,1,0
153,Whitecap Mountain,1750,400,1295,0,0.0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
154,Hogadon Basin,8000,640,7400,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
155,Sleeping Giant Ski Resort,7428,810,6619,0,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort, which you don't need in the dataframe for modeling so it can be droppped here as well. You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [6]:
# dropping Name column (name of the resort)
df.drop('Name',inplace=True, axis=1)

In [7]:
#standardizing the scale of the features of the dataframe columns, however not passing the dummy features to the 
#scaler as they as are boolean representations of categorical features
samples=df.iloc[:,:24]

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(samples)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
samples_scaled = scaler.transform(samples)

In [10]:
#scaled_df = pd.DataFrame(samples_scaled, columns=samples)

## Split into training and testing datasets

In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

**<font color='teal'> Using sklearn model selectionm import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [12]:
#Splitting the dataset into the Training and Test dataset
#train_set_split: Split arrays or matrices into random train and #test subsets. %25 of the dataset to the test set
X = df
y= df['AdultWeekend']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [13]:
len(X_train), len(X_test)

(117, 40)

In [14]:
X_train

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
30,790,262,528,0,0.0,0,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
26,7200,700,6500,0,0.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
93,3500,1100,2400,0,0.0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
146,1250,250,1000,0,0.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34,975,275,800,0,0.0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,7700,500,7200,0,0.0,0,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
83,9255,1055,8200,0,0.0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
153,1750,400,1295,0,0.0,0,0,1,0,4,...,0,0,0,0,0,0,0,0,1,0
58,1510,230,1280,0,0.0,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0


Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

In [15]:
#Fit multiple Linear Regression model to our Train set
from sklearn.linear_model import LinearRegression

In [16]:
#Create an object called regressor in the LinearRegression class
regressor = LinearRegression()

#Fit the linear regression model to the training set… We use the fit 
#method the arguments of the fit method will be training sets
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [17]:
y_pred = regressor.predict(X_test)

In [18]:
y_pred

array([64.16681004, 47.        , 58.        , 64.16681004, 70.        ,
       64.16681004, 47.        , 79.        , 39.        , 50.        ,
       68.        , 59.        , 79.        , 44.        , 37.        ,
       49.        , 60.        , 80.        , 75.        , 59.        ,
       72.        , 65.        , 69.        , 89.        , 93.        ,
       50.        , 44.        , 59.        , 78.        , 50.        ,
       47.        , 33.        , 55.        , 64.        , 45.        ,
       45.        , 65.        , 75.        , 55.        , 64.        ])

In [19]:
y_test

47     64.16681
25     47.00000
73     58.00000
119    64.16681
101    70.00000
28     64.16681
65     47.00000
133    79.00000
66     39.00000
54     50.00000
135    68.00000
90     59.00000
40     79.00000
150    44.00000
59     37.00000
48     49.00000
21     60.00000
84     80.00000
134    75.00000
8      59.00000
75     72.00000
44     65.00000
10     69.00000
16     89.00000
74     93.00000
125    50.00000
118    44.00000
70     59.00000
2      78.00000
72     50.00000
138    47.00000
116    33.00000
29     55.00000
100    64.00000
51     45.00000
77     45.00000
124    65.00000
89     75.00000
98     55.00000
144    64.00000
Name: AdultWeekend, dtype: float64

In [20]:
regressor.score(X_train, y_train)

1.0

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [21]:
#all first model set
from sklearn import linear_model, metrics
from sklearn.metrics import explained_variance_score,mean_absolute_error, r2_score
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [22]:
y_pred = lm.predict(X_test)

In [23]:
lm.score(X_test,y_test)

1.0

In [24]:
from math import sqrt
from sklearn.metrics import mean_squared_error

In [25]:
# measure Root Mean Squared Error
rmse = sqrt(mean_squared_error(y_test, y_pred))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', rmse)
print('R2 Value:', r2_score(y_test,y_pred))

Mean Absolute Error: 9.965361869035405e-14
Mean Squared Error: 1.8577989862316918e-26
Root Mean Squared Error: 1.3630110000406056e-13
R2 Value: 1.0


## Review Model Outcomes — Iterate over additional models as needed.

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
param_grid = {"n_estimators": [200, 500],
 "max_depth": [3, None],
 "max_features": [1, 3, 5, 10],
 "min_samples_split": [2, 5, 10],
 "min_samples_leaf": [1, 3, 10],
 "bootstrap": [True, False]}
model = RandomForestRegressor(random_state=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_params_)

0.8743769661636895
{'bootstrap': False, 'max_depth': None, 'max_features': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


**<font color='teal'> Print the intercept value from the linear model. </font>**

In [35]:
print('Slope of AdultWeekend',lm.coef_)
print('Intercept of AdultWeekend',lm.intercept_)

Slope of AdultWeekend [ 9.55457088e-17 -3.50630983e-16 -8.32667268e-17  2.63677968e-16
  2.77555756e-17  5.55111512e-17 -1.04916076e-14  2.30371278e-15
  6.77236045e-15 -2.57432964e-15  9.85322934e-16 -2.66453526e-15
 -1.66533454e-16  6.66133815e-16 -5.99520433e-15  1.86482774e-17
  1.29757316e-15 -1.98105421e-15  6.63531730e-17  4.99600361e-16
  1.73472348e-16  1.00000000e+00 -1.00700698e-15 -1.09807996e-15
  2.19529256e-15  1.15172628e-14  4.33680869e-18  4.66883476e-14
  5.71894962e-15 -5.31085592e-15  1.84323043e-14 -4.66710004e-14
 -3.69769319e-14 -8.44233537e-14 -2.54269262e-14 -8.67361738e-19
  2.21281327e-14  2.95371366e-14 -2.68418100e-14 -4.13668665e-14
  0.00000000e+00 -3.75580643e-14  4.00109633e-14  4.13191616e-14
 -2.71267384e-15 -3.34641169e-14 -2.53447437e-14 -1.63124722e-14
 -5.81132364e-17  9.93649607e-15  4.49902702e-14  0.00000000e+00
  9.43282995e-15  3.54083082e-14  2.44628536e-15  5.86369061e-14
 -3.86062710e-15 -1.11325879e-15  1.12284314e-14]
Intercept of Adult

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [28]:
params = pd.Series(lm.coef_, index=X.columns).abs().sort_values(ascending=False)
params

AdultWeekend            1.000000e+00
state_Iowa              8.442335e-14
state_Washington        5.863691e-14
state_California        4.668835e-14
state_Illinois          4.667100e-14
state_South Dakota      4.499027e-14
state_Missouri          4.136687e-14
state_New Mexico        4.131916e-14
state_New Hampshire     4.001096e-14
state_Nevada            3.755806e-14
state_Indiana           3.697693e-14
state_Vermont           3.540831e-14
state_North Carolina    3.346412e-14
state_Michigan          2.953714e-14
state_Minnesota         2.684181e-14
state_Maine             2.542693e-14
state_Ohio              2.534474e-14
state_Massachusetts     2.212813e-14
state_Idaho             1.843230e-14
state_Oregon            1.631247e-14
state_Alaska            1.151726e-14
state_Wyoming           1.122843e-14
fastQuads               1.049161e-14
state_Rhode Island      9.936496e-15
state_Utah              9.432830e-15
triple                  6.772360e-15
LongestRun_mi           5.995204e-15
s

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [29]:
df_mod2 = df.iloc[:,:25]
df_mod2

,summit_elev,vertical_drop,base_elev,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,cluster
0,2600,1540,1200,0,0.0,0,0,0,0,4,...,640.0,60.0,45.0,44.0,350.0,47.0,53.00000,90.0,0.0,0
1,2090,294,1796,0,0.0,0,0,0,1,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,0
2,11100,1800,9200,0,0.0,0,1,2,3,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,1
3,7800,600,7200,0,0.0,0,0,0,1,3,...,88.0,0.0,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,1
4,7700,500,7200,0,0.0,0,1,1,3,1,...,380.0,200.0,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,1160,300,860,0,0.0,0,0,0,3,0,...,32.0,32.0,112.0,61.0,41.0,17.0,48.00000,103.0,32.0,0
153,1750,400,1295,0,0.0,0,0,1,0,4,...,400.0,300.0,105.0,57.0,200.0,60.0,60.00000,118.0,0.0,0
154,8000,640,7400,0,0.0,0,0,0,0,1,...,92.0,32.0,121.0,61.0,80.0,48.0,48.00000,95.0,0.0,1
155,7428,810,6619,0,0.0,0,0,0,1,1,...,184.0,18.0,61.0,81.0,310.0,42.0,42.00000,77.0,0.0,1


In [30]:
X2 = df_mod2
y2= df_mod2['AdultWeekend']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,test_size=0.25)

In [31]:
lm2 = linear_model.LinearRegression()
model2 = lm2.fit(X2_train,y2_train)
y2_pred = lm2.predict(X2_test)
lm2.score(X2_test,y2_test)

1.0

In [32]:
# measure Root Mean Squared Error
rmse2 = sqrt(mean_squared_error(y2_test, y2_pred))

print('Mean Absolute Error:', metrics.mean_absolute_error(y2_test, y2_pred))
print('Mean Squared Error:', mean_squared_error(y2_test, y2_pred))
print('Root Mean Squared Error:', rmse2)
print('R2 Value:', r2_score(y2_test,y2_pred))

Mean Absolute Error: 6.10178574333986e-14
Mean Squared Error: 6.905688364304737e-27
Root Mean Squared Error: 8.310047150470771e-14
R2 Value: 1.0


In [33]:
print('Slope of AdultWeekend',lm2.coef_)
print('Intercept of AdultWeekend',lm2.intercept_)

Slope of AdultWeekend [-6.42659845e-17 -6.41847686e-17  8.02878262e-17 -2.42861287e-17
 -2.22044605e-16  5.55111512e-17  3.78789181e-15  1.58910779e-15
 -2.09615217e-15 -1.54782848e-15 -1.30329016e-15  8.11086548e-16
 -5.40360692e-17  3.02974851e-16 -1.26625386e-15  1.28073298e-16
 -1.17420498e-16 -1.00586857e-16 -2.17242775e-17 -9.76933920e-17
 -5.83350744e-16  1.00000000e+00  3.23693641e-16  6.47133172e-19
  9.17361828e-16]
Intercept of AdultWeekend 4.263256414560601e-14


In [34]:
params2 = pd.Series(lm2.coef_, index=X2.columns).abs().sort_values(ascending=False)
params2

AdultWeekend         1.000000e+00
fastQuads            3.787892e-15
triple               2.096152e-15
quad                 1.589108e-15
double               1.547828e-15
surface              1.303290e-15
LongestRun_mi        1.266254e-15
cluster              9.173618e-16
total_chairs         8.110865e-16
AdultWeekday         5.833507e-16
projectedDaysOpen    3.236936e-16
TerrainParks         3.029749e-16
fastEight            2.220446e-16
SkiableTerrain_ac    1.280733e-16
Snow Making_ac       1.174205e-16
daysOpenLastYear     1.005869e-16
averageSnowfall      9.769339e-17
base_elev            8.028783e-17
summit_elev          6.426598e-17
vertical_drop        6.418477e-17
fastSixes            5.551115e-17
Runs                 5.403607e-17
trams                2.428613e-17
yearsOpen            2.172428e-17
NightSkiing_ac       6.471332e-19
dtype: float64

**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection: